<a href="https://colab.research.google.com/github/clemsage/NeuralDocumentClassification/blob/master/skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Settting up the computing environment


## Install and import TensorFlow 2.0 with GPU

Select "GPU" in the Accelerator drop-down on Notebook Settings through the Edit menu.

In [0]:
!pip install tensorflow-gpu==2.0
import tensorflow as tf
print (tf.__version__)

## Confirm TensorFlow can see the GPU

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Additional information about hardware

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

For CPU information and RAM, run:

In [0]:
!cat /proc/cpuinfo
!cat /proc/meminfo

## Other useful package imports

In [0]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

# Working on the dataset

## Information about the dataset

In [0]:
class_names = ['email', 'form', 'handwritten', 'invoice', 'advertisement']
num_classes = len(class_names)

## Import the dataset

Import the dataset - a subset of the [RVL-CDIP dataset](https://www.cs.cmu.edu/~aharley/rvl-cdip/) - from the Google Drive [PROVIDE_LINK]

See [ways to import dataset in Google colab](https://medium.com/@master_yi/importing-datasets-in-google-colab-c816fc654f97)

In [0]:
num_examples = {"training": 100, "test": 10}
height_image, width_image = 800, 600

images, labels = dict(), dict()
for set_type, num_ex_set in num_examples.items():
  images[set_type] = np.random.randint(low=0, high=256, 
                                       size=(num_ex_set, height_image, width_image, 1))
  labels[set_type] = np.random.randint(low=0, high=num_classes, size=num_ex_set)

## Explore the data

Get the number of images in the training dataset:

In [0]:
print(len(images["training"]))

Get the width, height and depth of each image:

In [0]:
print(images["training"][0].shape)

Plot 5 random training images of each class:

In [0]:
plt.figure(figsize=(20, 10))
n_images_per_class = 5

for class_idx in range(num_classes):
  labels_idx = np.where(labels["training"] == class_idx)[0]
  np.random.shuffle(labels_idx)
  labels_idx = labels_idx[:n_images_per_class]
  
  for i in range(n_images_per_class):
    plt.subplot(num_classes, n_images_per_class, 
                class_idx*n_images_per_class + i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(np.squeeze(images["training"][labels_idx[i]]), cmap='gray')
    plt.xlabel(class_names[labels["training"][labels_idx[i]]])

plt.show()

Get the class distribution in the training set:

In [0]:
print({class_names[key]: val for key, val in Counter(labels["training"]).items()})

## Preprocess the data

Create a function called `preprocess_images` to reshape images to 299 x 299 pixels and scale the pixel values to the range [0, 1]

In [0]:
def preprocess_images(images, new_height_image=299, new_width_image=299):
  images = tf.image.resize(images, (new_height_image, new_width_image))
  images = images / 255.0
  return images

Verify the resulting image shape and pixel values

In [0]:
im = preprocess_images(images["training"][0])

print(im.shape)
print(np.min(im))
print(np.max(im))

# Visual classifiers

## Fully connected neural network

### Set up the layers

Build a neural network composed of one fully connected (aka dense) layer with 128 hidden units and one output layer.

Each image must be reshaped to a 1 dimensional vector before being fed to the hidden layer.

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(new_height_image, new_width_image, 1)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

### Compile the model

Compile the model by providing the optimizer, the loss function you want to minimize and the metrics to monitor during training

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train the model

Fit the model on the training preprocessed images for 10 epochs

In [0]:
train_images = preprocess_images(images["training"])
model.fit(train_images, labels["training"], epochs=10)